In [1]:
! pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
! pip install requests

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [3]:
! pip install lxml

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [4]:
! pip install folium

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [5]:
#import liabriaries

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# PART 1

**In the following code, we will read the html data from Wiki and clean the data.  The data can be found at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M** 

In [6]:
#import libraries
import pandas as pd
import numpy as np

In [7]:
#use pandas to read in the html info
# create a dataframe from the first table
data_toronto_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)[0]

In [8]:
data_toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# preserve the order of the columns
columns = list(data_toronto_df.columns)

#reset the index to Borough
data_toronto_df = data_toronto_df.set_index('Borough')

#delete all rows whose Borough = Not Assigned
data_toronto_df = data_toronto_df.drop("Not assigned", axis = 0)

# reinstate Borough as column
data_toronto_df = data_toronto_df.reset_index()


In [10]:
data_toronto_df.head()

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


In [11]:
#reinstate the orgignal order of the columns
data_toronto_df = data_toronto_df[columns]
data_toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [12]:
#define a function to deal with the "Not assigned" Neighbourhoods 

def fun_replace(row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    
    return row['Neighbourhood']

#replace the values of Not Assigned in the Neighbourhoods with the values of the Boroughs    
data_toronto_df['Neighbourhood'] = data_toronto_df.apply(fun_replace, axis = 1) 

In [13]:
#check there are no "Not assigned" neighborhoods
(data_toronto_df['Neighbourhood'] != 'Not assigned').all()

True

In [14]:
data_toronto_df.head(3)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront


In [15]:
data_toronto_grouped_df = data_toronto_df.groupby('Postcode', as_index = False).agg(lambda x: ', '.join(set(x)))

In [16]:
print("The number of rows in the cleaned datafrane is ", data_toronto_grouped_df.shape[0])

The number of rows in the cleaned datafrane is  103


# PART 2

In [17]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ #private data removed 

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_data_1.shape

(103, 3)

In [19]:
df_data_1 = df_data_1.rename(columns = {'Postal Code' : 'PostalCode'})
data_toronto_grouped_df = data_toronto_grouped_df.rename(columns = {'Postcode' : 'PostalCode'})

In [20]:
data_toronto_grouped_df = data_toronto_grouped_df.merge(df_data_1, on = 'PostalCode')

In [21]:
data_toronto_grouped_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# PART 3

In [22]:
neighbourhoods_of_Toronto_df = data_toronto_grouped_df[data_toronto_grouped_df['Borough'].str.contains('Toronto')]

In [23]:
neighbourhoods_of_Toronto_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Exploring venues in downtown Toronto

In [24]:
neighbourhoods_of_Downtown_Toronto_df = neighbourhoods_of_Toronto_df [neighbourhoods_of_Toronto_df['Borough'].str.contains('Downtown Toronto')]

In [25]:
neighbourhoods_of_Downtown_Toronto_df.reset_index(inplace = True, drop = True)
neighbourhoods_of_Downtown_Toronto_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, Richmond, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Toronto Islands, Union Station, Harbourfront East",43.640816,-79.381752


#### Use geopy library to get the latitude and longitude values of Downtown Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>dwntoro_explorer</em>, as shown below.

In [26]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="dwntoro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


#### Create a map of Downtown Toronto with neighborhoods superimposed on top.

In [44]:
# create map of Downtown Toronto using latitude and longitude values
map_dwnttoronto = folium.Map(location=[latitude, longitude], zoom_start=12, zoom_control = False)

n_dwnt_tor = neighbourhoods_of_Downtown_Toronto_df
# add markers to map
for lat, lng, borough, neighborhood in zip(n_dwnt_tor['Latitude'], n_dwnt_tor['Longitude'], n_dwnt_tor['Borough'], n_dwnt_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dwnttoronto)  
    
map_dwnttoronto

#### Use the Foursquare API to explore the neighborhoods and segment them.

In [28]:
CLIENT_ID = '....' # your Foursquare ID, private data removed
CLIENT_SECRET = '....' # your Foursquare Secret, private data removed
VERSION = '20190605' # Foursquare API version

Create a function to extract venues for all neighbourhoods of Downtown Toronto

In [29]:
# define LIMIT of the number of venues, set it at 50
# define radius from the center of Downtown Toronto, set it at 500 meters

LIMIT = 50
radius = 500

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:

dwnt_tor_data = neighbourhoods_of_Downtown_Toronto_df
dwntToronto_venues = getNearbyVenues(names=dwnt_tor_data['Neighbourhood'],
                                   latitudes=dwnt_tor_data['Latitude'],
                                   longitudes=dwnt_tor_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, Richmond, King
Toronto Islands, Union Station, Harbourfront East
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Grange Park, Chinatown, Kensington Market
Island airport, Harbourfront West, Railway Lands, CN Tower, Bathurst Quay, King and Spadina, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


Let's check how many venues were returned for each neighborhood

In [32]:
dwntToronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, Richmond, King",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
Central Bay Street,50,50,50,50,50,50
Christie,15,15,15,15,15,15
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Grange Park, Chinatown, Kensington Market",50,50,50,50,50,50
"Island airport, Harbourfront West, Railway Lands, CN Tower, Bathurst Quay, King and Spadina, South Niagara",15,15,15,15,15,15


##  Analyze Each Neighborhood

In [33]:
# one hot encoding
dwntToronto_venues_onehot = pd.get_dummies(dwntToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dwntToronto_venues_onehot['Neighbourhood'] = dwntToronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dwntToronto_venues_onehot.columns[-1]] + list(dwntToronto_venues_onehot.columns[:-1])
dwntToronto_venues_onehot = dwntToronto_venues_onehot[fixed_columns]

dwntToronto_venues_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping rows by neighbourhood  by taking the mean of the frequency of occurrence of each category

In [34]:
dwnttoronto_grouped = dwntToronto_venues_onehot.groupby('Neighbourhood').mean().reset_index()
dwnttoronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, Richmond, King",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.06,...,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.000000
1,Berczy Park,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.000000
2,Central Bay Street,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,...,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.02,0.00,0.020000
3,Christie,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000
4,Church and Wellesley,0.02,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.02,0.00,0.00,0.00,0.00,0.000000,0.02,0.00,0.02,0.000000
5,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.02,0.00,0.000000
6,"First Canadian Place, Underground city",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.02,0.00,0.000000
7,"Grange Park, Chinatown, Kensington Market",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.02,0.00,0.00,0.06,0.000000,0.04,0.02,0.00,0.000000
8,"Island airport, Harbourfront West, Railway Lan...",0.00,0.00,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000
9,"Regent Park, Harbourfront",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.020833


## Clustering Neighborhoods

The experiment is to cluster the neighbourhoods into few clusters.  While there are simpler ways to do it, the goal of this exercise is to use the clustering algorithm to visualize the neighborhoods of the downtown Toronto with the most venues, doesn't matter what they are.
The ultimate goal is to explore the zip codes (postal codes) belonging to the most populous cluster.
**Note:** The set up in the last part of the code is somewhat different from the set up of the Lab exercise for NY city venues

In [35]:

# set number of clusters
kclusters = 3

dwnttoronto_grouped_clustering = dwnttoronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dwnttoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
dwnttoronto_grouped_clustering.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,...,0.00,0.0,0.0,0.0,0.02,0.0,0.00,0.00,0.00,0.00
1,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.02,0.0,0.00,0.00,0.00,0.00
2,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,...,0.00,0.0,0.0,0.0,0.02,0.0,0.00,0.02,0.00,0.02
3,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00
4,0.02,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.02,0.0,0.0,0.0,0.00,0.0,0.02,0.00,0.02,0.00


In [37]:
dwnttoronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [38]:
dwnttoronto_combined_df = neighbourhoods_of_Downtown_Toronto_df.merge(dwnttoronto_grouped, on = 'Neighbourhood')

In [39]:
columns_list = ['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Cluster Labels']
dwnttoronto_combined_df = dwnttoronto_combined_df[columns_list]
dwnttoronto_combined_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0


## Map clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10, zoom_control = False)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dwnttoronto_combined_df['Latitude'], dwnttoronto_combined_df['Longitude'], dwnttoronto_combined_df['Neighbourhood'], dwnttoronto_combined_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Analyze the neighbourhoods to determine the most populous cluster

In [41]:
dwnttoronto_combined_df_pop = dwnttoronto_combined_df.groupby('Cluster Labels').count()

In [42]:
dwnttoronto_combined_df_pop

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
Cluster Labels,,,,,
0,16,16,16,16,16
1,1,1,1,1,1
2,1,1,1,1,1


**The Conclusion**: The most venues belong to the zeros cluster based on k-means algorithm